In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import re
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex

Загрузка датасета

In [6]:
train = pd.read_csv("additional_data/building_20230808.csv")[['id', 'full_address']]
train

/tmp/ipykernel_10063/2992611257.py:1: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("additional_data/building_20230808.csv")[['id', 'full_address']]


,id,full_address
0,56343,"город Пушкин, Кедринская улица, дом 12"
1,595,"поселок Ушково, Пляжевая улица, дом 4Б"
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, до..."
3,124415,"поселок Белоостров, Дюны, Центральная улица, д..."
4,185368,"поселок Песочный, Речная улица, дом 28, литера Б"
...,...,...
166640,18941,"г.Санкт-Петербург, Бородинская улица, дом 3, л..."
166641,18942,"посёлок Александровская, 2-я линия, дом 34/3, ..."
166642,18943,"посёлок Ушково, Детская улица, дом 39"
166643,18944,"г.Санкт-Петербург, Заусадебная улица, дом 25, ..."


Получение векторов адресов

In [3]:
model = SentenceTransformer('intfloat/multilingual-e5-large')

In [7]:
train['embeding']=train['full_address'].progress_apply(lambda x: model.encode(x, convert_to_tensor=False))

100%|███████████████████████████████████| 166645/166645 [30:54<00:00, 89.85it/s]


In [8]:
train.head()

,id,full_address,embeding
0,56343,"город Пушкин, Кедринская улица, дом 12","[0.0230154, 0.008957184, -0.030653214, -0.0420..."
1,595,"поселок Ушково, Пляжевая улица, дом 4Б","[0.025424432, 0.0071238964, -0.017093355, -0.0..."
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, до...","[0.0038483946, -0.022700313, -0.027302384, -0...."
3,124415,"поселок Белоостров, Дюны, Центральная улица, д...","[0.025931468, 0.013823913, -0.048282415, -0.03..."
4,185368,"поселок Песочный, Речная улица, дом 28, литера Б","[0.023821864, -0.019470476, -0.018878864, -0.0..."


Построение индекса

In [10]:
#"angular", "euclidean", "manhattan", "hamming", or "dot" 384 768 1024
t = AnnoyIndex(1024, metric = 'angular')

# train['id'], 
for user_id, user_embedding in enumerate(train['embeding']):
    t.add_item(user_id, user_embedding)
t.build(-1)

True

In [11]:
def predict_nns(x):
    idx = t.get_nns_by_vector(x, 1, search_k=-1)  
    return train['id'][idx].values[0]

Валидация

In [12]:
test = pd.read_csv("test_example.csv", sep=";")
test

,id,address,target_building_id
0,0,"197046, Санкт-Петербург (г.), Куйбышева (ул.),...",67206
1,1,"197101, Санкт-Петербург (г.), Красного Курсант...",48868
2,2,"197136, Санкт-Петербург г, Ленина ул, д.20",66919
3,3,"197022, Санкт-Петербург (г.), Санкт-Петербург ...",53075
4,4,"197022, Санкт-Петербург (г.), Санкт-Петербург ...",13759
...,...,...,...
323,323,"Россия, Санкт-Петербург, проспект Энгельса, 134к3",107461
324,324,"Россия, Санкт-Петербург, проспект Кузнецова, 21",34193
325,325,"Россия, Санкт-Петербург, проспект Ветеранов, 78",111811
326,326,"Россия, Санкт-Петербург, Ленская улица, 6к1",23918


In [13]:
#|улица|проспект|литера|строение
test['address']=test['address'].apply(lambda x: re.sub(r"\b(\d{6})\b|Россия", "", x, flags=re.IGNORECASE))

In [14]:
test['embeding']=test['address'].progress_apply(lambda x: model.encode(x, convert_to_tensor=False))

100%|█████████████████████████████████████████| 328/328 [00:03<00:00, 93.60it/s]


In [15]:
test["predict"]=test["embeding"].progress_apply(lambda x: predict_nns(x))

100%|███████████████████████████████████████| 328/328 [00:00<00:00, 1197.51it/s]


In [16]:
test

,id,address,target_building_id,embeding,predict
0,0,", Санкт-Петербург (г.), Куйбышева (ул.), д.1/5...",67206,"[0.007560225, 0.008159104, -0.040068414, -0.03...",67206
1,1,", Санкт-Петербург (г.), Красного Курсанта (ул....",48868,"[0.008343507, -0.011526222, -0.03700694, -0.05...",48868
2,2,", Санкт-Петербург г, Ленина ул, д.20",66919,"[-0.001104841, -0.0030358918, -0.05111607, -0....",66919
3,3,", Санкт-Петербург (г.), Санкт-Петербург г, Орд...",53075,"[0.005841343, -0.013016736, -0.0508436, -0.052...",9118
4,4,", Санкт-Петербург (г.), Санкт-Петербург г, Орд...",13759,"[0.0014988815, -0.010564016, -0.045057923, -0....",13759
...,...,...,...,...,...
323,323,", Санкт-Петербург, проспект Энгельса, 134к3",107461,"[0.010136293, 0.005330934, -0.009662743, -0.03...",107461
324,324,", Санкт-Петербург, проспект Кузнецова, 21",34193,"[0.0020240585, -0.00967046, -0.032553103, -0.0...",34193
325,325,", Санкт-Петербург, проспект Ветеранов, 78",111811,"[0.020891342, -0.03149204, -0.031932812, -0.06...",90950
326,326,", Санкт-Петербург, Ленская улица, 6к1",23918,"[0.02268179, 0.015187119, -0.025827594, -0.047...",23918


Метрика

In [17]:
accuracy_score(test['target_building_id'], test['predict'])

0.21951219512195122

In [60]:
#сохранение модели
#t.save('train.ann')

True

In [61]:
#загрузка модели
#t = AnnoyIndex(384, 'angular')
#t.load('train.ann') 

True

In [78]:
test[test['target_building_id']==test['predict']]

,id,address,target_building_id,embeding,predict
4,4,"197022, Санкт-Петербург (г.), Санкт-Петербург ...",13759,"[0.015561515, -0.0022936119, -0.014905811, -0....",13759
26,26,"197371, Санкт-Петербург г, Королева пр-кт, д.4...",15929,"[0.025195858, 0.034265913, -0.025032409, 0.008...",15929
106,106,"Ветеранов, д.71",5511,"[0.038207497, 0.046261165, -0.036857672, -0.04...",5511
198,198,"Автовская, д.5",111131,"[0.035692, -0.010759905, -0.016322404, -0.0796...",111131
221,221,"Бульвар Новаторов, д.58",2248,"[0.005473601, 0.006582194, -0.03595792, -0.059...",2248
249,249,"Большой пр В.О., д.18",97617,"[0.013459695, 0.008548084, -0.039983258, -0.02...",97617
255,255,"8 Красноармейская, д.14",20282,"[0.047296092, -0.005002274, -0.023894833, -0.0...",20282
260,260,"15 линия, д.28Г",63897,"[0.012776611, 0.017596612, -0.062928595, -0.04...",63897
266,266,"Будапештская, д.66",98094,"[0.01538562, 0.0029835452, -0.031971462, -0.06...",98094
280,280,"Россия, Санкт-Петербург, Муринская дорога, 59к1",214968,"[0.036834303, -0.06211141, -0.024646407, 0.009...",214968


Тестирование

In [18]:
submission = pd.read_csv("test.csv", sep=";")
submission

,id,address,target_building_id
0,0,"190121, Санкт-Петербург (г.), Санкт-Петербург ...",0
1,1,"196084, Санкт-Петербург (г.), Санкт-Петербург,...",1
2,2,"190068, Санкт-Петербург г, Садовая ул, д.60",2
3,3,"190103, Санкт-Петербург (г.), Санкт-Петербург ...",3
4,4,"190103, Санкт-Петербург г, Лермонтовский пр-кт...",4
...,...,...,...
2925,2925,пр. Шуваловский 32 к.3 лит.А,2925
2926,2926,Шуваловский проспект 32к3 А,2926
2927,2927,Iedfkjdcrbq ghjcgtrn 32 rjhgec 3 kbnthf F,2927
2928,2928,Uhbdwjdf 22,2928


In [19]:
submission['address']=submission['address'].apply(lambda x: re.sub(r"\b(\d{6})\b|Россия", "", x, flags=re.IGNORECASE))
submission['embeding']=submission['address'].progress_apply(lambda x: model.encode(x, convert_to_tensor=False))

100%|███████████████████████████████████████| 2930/2930 [00:30<00:00, 95.75it/s]


In [20]:
submission["target_building_id"]=submission["embeding"].progress_apply(lambda x: predict_nns(x))

100%|█████████████████████████████████████| 2930/2930 [00:01<00:00, 1531.22it/s]


In [21]:
submission.head()

,id,address,target_building_id,embeding
0,0,", Санкт-Петербург (г.), Санкт-Петербург г, наб...",51633,"[0.008146789, -0.01916938, -0.02385905, -0.057..."
1,1,", Санкт-Петербург (г.), Санкт-Петербург, Парфе...",212853,"[0.0035544843, -0.01141959, -0.014238481, -0.0..."
2,2,", Санкт-Петербург г, Садовая ул, д.60",24014,"[0.0007633302, -0.015093371, -0.023972813, -0...."
3,3,", Санкт-Петербург (г.), Санкт-Петербург г, Кра...",78683,"[-0.0029713465, -0.02456147, -0.052630488, -0...."
4,4,", Санкт-Петербург г, Лермонтовский пр-кт, д.49",23269,"[-0.011129197, -0.018168291, -0.040407613, -0...."


In [24]:
submission[["id", "target_building_id"]].to_csv("submission1.csv", sep=';', index=False)